# Setup

In [ ]:
!pip install --upgrade google-cloud-aiplatform google-cloud-logging --quiet
!pip install "google-cloud-aiplatform[evaluation]" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 25.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import logging
import google.cloud.logging
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PointwiseMetric,
)

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

pd.set_option("display.max_colwidth", None)

In [2]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'NV_LIBCUBLAS_VERSION': '12.5.3.2-1',
        'NVIDIA_VISIBLE_DEVICES': 'all',
        'NV_NVML_DEV_VERSION': '12.5.82-1',
        'NV_CUDNN_PACKAGE_NAME': 'libcudnn9-cuda-12',
        'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.22.3-1+cuda12.5',
        'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.22.3-1',
        'VERTEX_PRODUCT': 'COLAB_ENTERPRISE',
        'VM_GCE_METADATA_HOST': '169.254.169.254',
        'HOSTNAME': '6e016d51ce48',
        'LANGUAGE': 'en_US',
        'COLAB_TPU_1VM': '',
        'GOOGLE_CLOUD_PROJECT': 'qwiklabs-gcp-01-ecf3a270a90b',
        'GCE_METADATA_TIMEOUT': '3',
        'NVIDIA_REQUIRE_CUDA': 'cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand

In [3]:
PROJECT_ID = "qwiklabs-gcp-01-ecf3a270a90b"
LOCATION = "us-west1"

import vertexai

# Initialize vertexai
vertexai.init()

# Do not remove logging section
log_message = f"Vertex AI initialize: {vertexai}"
logging.info(log_message)

INFO:root:Vertex AI initialize: <module 'vertexai' from '/usr/local/lib/python3.11/dist-packages/vertexai/__init__.py'>


# Task 2. Explore example data and make a simple summarization request to a Gen AI model

In this task, you will set up some sample product description data for a large language model to summarize and then use a prompt to send a simple summarization request to the model. This configuration will then be used for evaluating the models performance in providing summarizations.



In [5]:
# The prompt you will use for summarization is shown below. Run the following code in a new cell to set the prompt_template.

prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}"


In [6]:
# Next, define the system prompt and question for the prompt template and some sample product data to be summarised.

system_prompt=["You are an retail domestic merchandise expert"]

question=["Provide a one sentence summary of the following text"]

description=[
  "Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated",

  "Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.",

  "Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats.",

  "OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece.",

  "Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and water. Whether it’s post-dinner chaos or a pile of party plates, this dishwasher handles it all with precision and polish. With advanced spray technology, high-temperature sanitization, and intuitive smart cycles, every dish comes out sparkling — from delicate wine glasses to stubborn, baked-on pots. The sleek stainless steel finish adds a modern touch to any kitchen, while the adjustable racks and spacious interior let you load more in fewer cycles. Plus, with smart connectivity, you can monitor and control your wash right from your phone — because clean dishes should never slow you down. "
]

# Make sure that all arrays (e.g., system_prompt, question, description) are of the same length to avoid errors when populating the DataFrame.

In [8]:
# Instantiate a flash_model variable with a GenerativeModel that uses Gemini version gemini-2.0-flash.
# Add a generation configuration to the model to set the temperature to 0.

generation_config = GenerationConfig(temperature=0)
flash_model = GenerativeModel("gemini-2.0-flash",
                              generation_config=generation_config)


In [10]:
# Using the prompt template and data,
# generate a response to the second product description (description[1]) from the gemini-2.0-flash model.
# Use the Markdown() class imported from IPython.display to wrap the response
# text to render Gemini's responses, which are often formatted as Markdown strings.


llm_response = flash_model.generate_content(
  # Fill in the appropriate configuration
  prompt_template.format(
      system_prompt=system_prompt[0],
      question=question[0],
      # select only: "summer floral midi dress"
      description=description[1]
  )
)

Markdown(llm_response.text)

# Do not remove logging section
log_message = f"Markdown output: {llm_response.text}"
logging.info(log_message)

INFO:root:Markdown output: This summer floral midi dress offers effortless elegance with its lightweight fabric, vibrant floral print, flattering silhouette, and delicate details, perfect for both casual and dressy occasions.



# Task 3. Prepare the Evaluation Dataset, EvalTask and evaluate of a single model

In this task, you will set up the data and scoring method to evaluate the model.

You will evaluate the models' summarization response by using Pointwise summarization quality.

https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#pointwise_summarization_quality


In [12]:
len(system_prompt), len(question), len(description)

(1, 1, 5)

In [13]:
system_prompt*5

['You are an retail domestic merchandise expert',
 'You are an retail domestic merchandise expert',
 'You are an retail domestic merchandise expert',
 'You are an retail domestic merchandise expert',
 'You are an retail domestic merchandise expert']

In [14]:
# Prepare a Pandas DataFrame with the fields needed to populate the prompt_template.
# dataset=pd.DataFrame(
#     {
#       # Fill in the appropriate configuration
#     }
# )

dataset = pd.DataFrame(
    {
      "system_prompt": system_prompt*len(description),
      "question": question*len(description),
      "description": description,
    }
)

display(dataset)

,system_prompt,question,description
0,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated"
1,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments."
2,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats."
3,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece."
4,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and water. Whether it’s post-dinner chaos or a pile of party plates, this dishwasher handles it all with precision and polish. With advanced spray technology, high-temperature sanitization, and intuitive smart cycles, every dish comes out sparkling — from delicate wine glasses to stubborn, baked-on pots. The sleek stainless steel finish adds a modern touch to any kitchen, while the adjustable racks and spacious interior let you load more in fewer

In [19]:
# Create an EvalTask(), passing in the dataset,
# identifying MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY
# as the metric you would like to be calculated,
POINTWISE_METRIC = PointwiseMetric(
    # Fill in the appropriate configuration
    metric="summarization_quality",
    metric_prompt_template=MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY
)


# and defining an experiment name of product-summarization-quality.
pointwise_eval_task = EvalTask(
    # Fill in the appropriate configuration
    dataset=dataset,
    # METRIC IN the TASK
    metrics=[POINTWISE_METRIC],
    experiment="product-summarization-quality",
)


# Run the evaluation of the EvalTask you configured above.
pointwise_result = pointwise_eval_task.evaluate(
    model=flash_model, #YOUR_MODEL_VARIABLE,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
    # experiment_run_name="product-summarization-quality",
)

INFO:root:MetadataStore default not found.


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n{system_prompt} # Question\n{question} # Description {description}', 'model_name': 'publishers/google/models/gemini-2.0-flash'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-2.0-flash.
100%|██████████| 5/5 [00:00<00:00,  7.84it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-2.0-flash.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 0.6556279830001586 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 5 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 5/5 [00:22<00:00,  4.43s/it]
INFO:vertexai.evaluation._eva

In [20]:
# Use the result of the evaluation to display the summary_metrics and metrics_table properties of the result.
eval_results_to_compare = []
eval_results_to_compare.append(pointwise_result)
eval_results_to_compare

[EvalResult(summary_metrics={'row_count': 5, 'summarization_quality/mean': np.float64(-0.6), 'summarization_quality/std': 0.5477225575051662}, metrics_table=                                   system_prompt  \
 0  You are an retail domestic merchandise expert   
 1  You are an retail domestic merchandise expert   
 2  You are an retail domestic merchandise expert   
 3  You are an retail domestic merchandise expert   
 4  You are an retail domestic merchandise expert   
 
                                                question  \
 0  Provide a one sentence summary of the following text   
 1  Provide a one sentence summary of the following text   
 2  Provide a one sentence summary of the following text   
 3  Provide a one sentence summary of the following text   
 4  Provide a one sentence summary of the following text   
 
                                                                                                                                                                  

In [21]:
pointwise_result.summary_metrics

{'row_count': 5,
 'summarization_quality/mean': np.float64(-0.6),
 'summarization_quality/std': 0.5477225575051662}

In [22]:
pointwise_result.metrics_table

,system_prompt,question,description,prompt,response,summarization_quality/explanation,summarization_quality/score
0,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","# System_prompt\nYou are an retail domestic merchandise expert # Question\nProvide a one sentence summary of the following text # Description Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","These men's blue dress shorts offer a refined, comfortable style suitable for various smart-casual occasions with their slim fit and breathable cotton-blend fabric.\n",I cannot provide a score or explanation for summarization quality without the original text and the summary to evaluate.,0.0
1,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","# System_prompt\nYou are an retail domestic merchandise expert # Question\nProvide a one sentence summary of the following text # Description Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","This summer floral midi dress offers effortless elegance with its lightweight fabric, vibrant floral print, flattering silhouette, and delicate details, perfect for both casual and dressy occasions.\n",Cannot evaluate summarization quality as the original text and the summary to be evaluated were not provided in the prompt.,-1.0
2,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Outdoor Ga

In the evaluation service results metric table which of the following are used in explaining the summarization/quality score? (Select all that apply)
- YES Grounded
- YES Concise
- YES Follows the instructions
- YES Fluent

# Task 4. Evaluate and compare 2 models
In this task, you will compare the summarization ability of gemini-2.0-flash and gemini-2.0-flash-lite models which can be used in model selection.

You will evaluate the models' responses against each other by using Pairwise summarization quality. You will use the gemini-2.0-flash-lite responses as your baseline model response and your gemini-2.0-flash responses as your candidate responses.

https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#pairwise_summarization_quality



In [23]:
# You will use the same Pandas DataFrame as for the single model evaluation.
# Create an EvalTask() instance, passing in the dataset, identifying pairwise_summarization_quality as the prompt template you would like to retrieve from MetricPromptTemplateExamples, and defining an experiment named pairwise-product-summarization-quality.
from vertexai.evaluation import MetricPromptTemplateExamples


generation_config = GenerationConfig(temperature=0)

# Instantiate the baseline model
baseline_model = GenerativeModel("gemini-2.0-flash-lite",
                                 generation_config=generation_config)

# Create the PairwiseMetric instance
# Fill in the configuration to load
# the prompt template from the examples
PAIRWISE_METRIC = PairwiseMetric(
    # Include your baseline model for pairwise comparison
    baseline_model=baseline_model,   #YOUR_BASELINE_MODEL,
    metric="summarization_quality",
    metric_prompt_template=MetricPromptTemplateExamples.Pairwise.SUMMARIZATION_QUALITY
)


# Create the EvalTask instance
# Fill in the appropriate configuration
pairwise_eval_task = EvalTask(
    dataset=dataset,
    metrics=[PAIRWISE_METRIC],
    experiment="pairwise-product-summarization-quality",
)

# Instantiate the candidate model
candidate_model = GenerativeModel("gemini-2.0-flash",
                                  generation_config=generation_config)

# Run the evaluation
# Run the evaluation of the EvalTask you configured above.
pairwise_result = pairwise_eval_task.evaluate(
    model=candidate_model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
    experiment_run_name="gemini-flash-vs-flash-lite",
)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n{system_prompt} # Question\n{question} # Description {description}', 'model_name': 'publishers/google/models/gemini-2.0-flash'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-2.0-flash.
100%|██████████| 5/5 [00:00<00:00,  5.94it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-2.0-flash.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 0.8484921049998775 seconds.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-2.0-flash-lite.
100%|██████████| 5/5 [00:00<00:00,  7.11it/s]
INFO:vertexai.evaluation._evaluation:

In [24]:
eval_results_to_compare.append(pairwise_result)

In [25]:
# Print the summary_metrics of the evaluation results.
display(pairwise_result.summary_metrics)

{'row_count': 5,
 'summarization_quality/candidate_model_win_rate': np.float64(0.4),
 'summarization_quality/baseline_model_win_rate': np.float64(0.2)}

In [26]:
# Display the evaluation response's metrics_table. Do you see a clear preference for either model, according to the evaluation service?
display(pairwise_result.metrics_table)

,system_prompt,question,description,prompt,response,baseline_model_response,summarization_quality/explanation,summarization_quality/pairwise_choice
0,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","# System_prompt\nYou are an retail domestic merchandise expert # Question\nProvide a one sentence summary of the following text # Description Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","These men's blue dress shorts offer a refined, comfortable style suitable for various smart-casual occasions with their slim fit and breathable cotton-blend fabric.\n","These men's blue dress shorts offer a comfortable, slim-fit design in a breathable cotton blend, perfect for smart-casual occasions.\n","No summaries (A) or (B) were provided for comparison, so a judgment cannot be made. Assuming they are equally missing.",TIE
1,You are an retail domestic merchandise expert,Provide a one sentence summary of the following text,"Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","# System_prompt\nYou are an retail domestic merchandise expert # Question\nProvide a one sentence summary of the following text # Description Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","This summer floral midi dress offers a cool, confident, and feminine look with its lightweight fabric, vibrant floral print, flattering silhouette, and delicate details, perfect for both daytime and evening wear.\n","This summer floral midi dress offers a lightweight, bre

In [30]:
pairwise_result.metrics_table.columns

Index(['system_prompt', 'question', 'description', 'prompt', 'response',
       'baseline_model_response', 'summarization_quality/explanation',
       'summarization_quality/pairwise_choice'],
      dtype='object')

In [31]:
# To simplify reading the results, display the column from the metrics_table that includes the evaluation service's preferred response for this example.
# Next, display the column from the metrics_table that contains the model's explanations of its choices. Read some of the results to understand the evaluation service's preferences.

display(pairwise_result.metrics_table["summarization_quality/pairwise_choice"])
display(pairwise_result.metrics_table["summarization_quality/explanation"])

,summarization_quality/pairwise_choice
0,TIE
1,TIE
2,BASELINE
3,CANDIDATE
4,CANDIDATE


,summarization_quality/explanation
0,"No summaries (A) or (B) were provided for comparison, so a judgment cannot be made. Assuming they are equally missing."
1,I need the actual summaries (A) and (B) to evaluate and compare their quality. Please provide the summaries.
2,"Summary A is more concise and avoids including extra details like the exact number of people in the survey or listing specific symptoms as extensively as summary B, while still capturing the main points."
3,"Summary B offers a slightly more active voice, better cohesion with 'This misinformation,' and slightly varied vocabulary, making it marginally more refined than A while conveying the same information."
4,"Summary B includes slightly more specific details from the article, such as competitiveness with fossil fuels and electric vehicles, while maintaining conciseness and accuracy, making it marginally more comprehensive than A."


In [33]:
pairwise_result.metrics_table[["summarization_quality/pairwise_choice", "summarization_quality/explanation", "baseline_model_response", "response"]]

,summarization_quality/pairwise_choice,summarization_quality/explanation,baseline_model_response,response
0,TIE,"No summaries (A) or (B) were provided for comparison, so a judgment cannot be made. Assuming they are equally missing.","These men's blue dress shorts offer a comfortable, slim-fit design in a breathable cotton blend, perfect for smart-casual occasions.\n","These men's blue dress shorts offer a refined, comfortable style suitable for various smart-casual occasions with their slim fit and breathable cotton-blend fabric.\n"
1,TIE,I need the actual summaries (A) and (B) to evaluate and compare their quality. Please provide the summaries.,"This summer floral midi dress offers a lightweight, breathable, and flattering silhouette with a vibrant floral print, perfect for various warm-weather occasions.\n","This summer floral midi dress offers a cool, confident, and feminine look with its lightweight fabric, vibrant floral print, flattering silhouette, and delicate details, perfect for both daytime and evening wear.\n"
2,BASELINE,"Summary A is more concise and avoids including extra details like the exact number of people in the survey or listing specific symptoms as extensively as summary B, while still capturing the main points.","This outdoor garden furniture set offers a comfortable, durable, and stylish solution for creating a relaxing outdoor space, suitable for various settings and weather conditions.\n","This outdoor garden furniture set offers a durable, weather-resistant, and stylish way to transform any outdoor space into a comfortable oasis for relaxation and entertainment.\n"
3,CANDIDATE,"Summary B offers a slightly more active voice, better cohesion with 'This misinformation,' and slightly varied vocabulary, making it marginally more refined than A while conveying the same information.","This OLED 4K Ultra HD Smart TV offers a cinematic home entertainment experience with breathtaking clarity, vibrant color, and immersive sound, enhanced by AI processing and a sleek design.\n","This OLED 4K Ultra HD Smart TV offers breathtaking clarity, vibrant color, and cinematic sound, transforming your living room into a private theater with lifelike detail and stunning contrast.\n"
4,CANDIDATE,"Summary B includes slightly more specific details from the article, such as competitiveness with fossil fuels and electric vehicles, while maintaining conciseness and accuracy, making it marginally more comprehensive than A.","The SmartWash Dishwasher offers a powerful, efficient, and smart cleaning experience with advanced features and smart connectivity for sparkling dishes.\n","The SmartWash Dishwasher offers a powerful, efficient, and convenient cleaning solution for all types of dishes, with smart features and a sleek design to simplify your kitchen routine.\n"


**Student explanation**
- From the table above, I am wondering why the Google evaluation service did not parse the summaries for the 1st 2 rows
- For record 3 on the outdoor garden furniture set, the Google evaluation service was hallucinating about 'exact number of people' inside 'candidate response'/summary B, when it does not exist
- For record 4 on OLED Smart TV, Google evaluation service talked about misinformation in the 'candidate response'/summary B when there wasn't any
- For record 5 on smartwash dishwasher, Google evaluation service was hallucinating about electric vehicles and fossil fuels
- **This set of evaluation results are not reliable and make me want to stay away from using 'Google evaluation service' for now, until the hallucinations are fixed**